<a href="https://colab.research.google.com/github/solcanalla/fiumark/blob/main/knn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Modelo: KNN #


##Inicialización##

In [ ]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import matplotlib.patches as mpatches
import matplotlib
import seaborn as sns
from sklearn.metrics import accuracy_score
from sklearn.neighbors import KNeighborsClassifier

##Pre procesamiento##

In [ ]:
#muy suceptible al ruido
!git clone https://github.com/solcanalla/fiumark.git
%cd fiumark
!git pull origin main
import preprocessing as pp

fatal: destination path 'fiumark' already exists and is not an empty directory.
/content/fiumark
From https://github.com/solcanalla/fiumark
 * branch            main       -> FETCH_HEAD
Already up to date.


Para KNN es importante escalar los datos. 

In [ ]:
pd.options.display.max_columns = None

df = pp.get_dataset()
ohe = pp.get_ohe_fit(df)
X_train, X_test, y_train, y_test = pp.get_train_test_data(df)
X_train = pp.knn_preprocessing(X_train,ohe)


##Entrenamiento##

In [ ]:
from sklearn.model_selection import RandomizedSearchCV

params = {
    'n_neighbors': np.arange(1, 100),
    'weights': ['uniform', 'distance'],
    'metric': ['cosine','minkowski'],
    'algorithm':['ball_tree','kd_tree','brute']
}

clf = KNeighborsClassifier()

rgscv = RandomizedSearchCV(
    clf, params, n_iter=200, scoring='accuracy', n_jobs=-1, cv=5, return_train_score=True
).fit(X_train, y_train)

In [ ]:
print(f"Best score: {rgscv.best_score_}")
print(f"Best params {rgscv.best_params_}")

Best score: 0.8102941176470587
Best params {'weights': 'uniform', 'n_neighbors': 20, 'metric': 'minkowski', 'algorithm': 'ball_tree'}


##Evaluación del modelo##

In [ ]:
from sklearn.metrics import classification_report

X_test = pp.knn_preprocessing(X_test,ohe)
print(classification_report(y_test, rgscv.predict(X_test)))

              precision    recall  f1-score   support

           0       0.77      0.96      0.85        70
           1       0.91      0.61      0.73        51

    accuracy                           0.81       121
   macro avg       0.84      0.78      0.79       121
weighted avg       0.83      0.81      0.80       121



##Predicción##

In [ ]:
#File for preprocessing data
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import (
	LabelEncoder,
	OneHotEncoder,
	Normalizer,
	MinMaxScaler
)

In [ ]:
def common_preprocessing(df):
	#Reseteo el indice por si entro con el train 
	df.reset_index(inplace=True,drop=True)
	ohe = OneHotEncoder(drop='first')
	ohe = ohe.fit(df[['genero','fila','nombre_sede','tipo_de_sala']].astype(str))
	#Clean
	del df['nombre']

	#Missing values
	#edad 20%
	#fila 80%
	#nombre_sede 0.2%

	median_imputer = SimpleImputer(strategy='median')
	edad_filled = median_imputer.fit_transform(df[['edad']])
	filled = pd.DataFrame(edad_filled).add_prefix('edad_')
	df = pd.concat([df, filled], axis=1)
	del df['edad']

	#Categoricas con baja cardinalidad
	#genero, nombre de sede, tipo de sala y fila.

	#Encondeo sin orden
	matrix_result = ohe.transform(df[['genero','fila','nombre_sede','tipo_de_sala']].astype(str)).todense().astype(int)
	df = pd.concat([df, pd.DataFrame(matrix_result)], axis=1)

	del df['genero']
	del df['nombre_sede']
	del df['tipo_de_sala']
	del df['fila']

	#Categóricas de alta cardinalidad
	del df['id_ticket']

	return df

In [ ]:
def knn_preprocessing(X):
	X = common_preprocessing(X) #Si normalizo tengo 0.7
	X = MinMaxScaler().fit_transform(X)
	return X

In [ ]:
X_holdout = pd.read_csv('https://drive.google.com/uc?export=download&id=1I980-_K9iOucJO26SG5_M8RELOQ5VB6A')
X = knn_preprocessing(X_holdout)
df2 = pd.DataFrame(rgscv.predict(X))
df3 = pd.concat([X_holdout.id_usuario,df2],axis=1)
df3.to_csv('knn.csv')
df3

,id_usuario,0
0,650,0
1,234,0
2,31,0
3,552,0
4,411,0
...,...,...
85,354,0
86,82,0
87,172,0
88,8,0
